Dataset cargado. Dimensiones: (116203, 13)


Características seleccionadas: ['Elevation', 'Slope', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Horizontal_Distance_To_Fire_Points']


RuntimeError: Files in same split ./data/covertype/* have different header.

In [34]:
#%% 4.3 - StatisticsGen (Estadísticas)
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])  # Visualizar en FACETS

ValueError: MetadataStore is expecting either proto.ConnectionConfig or proto.MetadataStoreClientConfig

In [35]:
#%% 4.4 - SchemaGen (Esquema Inferido)
from tfx.components import SchemaGen

schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])  # Mostrar esquema

ValueError: MetadataStore is expecting either proto.ConnectionConfig or proto.MetadataStoreClientConfig

In [36]:
#%% 4.5 - Curar Esquema con TFDV
import tensorflow_data_validation as tfdv

# Cargar esquema inferido
schema = tfdv.load_schema_text(schema_gen.outputs['schema'].get()[0].uri)

# Definir dominios y tipos
for feature in ['Hillshade_9am', 'Hillshade_Noon']:
    tfdv.set_domain(schema, feature, min=0, max=255)
tfdv.set_domain(schema, 'Slope', min=0, max=90)
tfdv.set_domain(schema, 'Cover_Type', min=0, max=6, is_categorical=True)

# Guardar esquema actualizado
tfdv.write_schema_text(schema, './schema/curated_schema.pbtxt')

IndexError: list index out of range

In [37]:
#%% 4.6 - Entornos de Esquema (Training/Serving)
# Crear dataset de servicio sin etiqueta
serving_data = df.drop(columns=['Cover_Type']).sample(100)
serving_data.to_csv(os.path.join(_data_root, 'serving.csv'), index=False)

# Configurar entornos
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
tfdv.write_schema_text(schema, './schema/schema_with_envs.pbtxt')

In [38]:
#%% 4.7 - Recalcular Estadísticas con Esquema
from tfx.components import ImporterNode

schema_importer = ImporterNode(
    instance_name='schema_importer',
    source_uri='./schema',
    artifact_type=standard_artifacts.Schema
)
context.run(schema_importer)

updated_stats_gen = StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result']
)
context.run(updated_stats_gen)

ImportError: cannot import name 'ImporterNode' from 'tfx.components' (/opt/venv/lib/python3.9/site-packages/tfx/components/__init__.py)

In [39]:
#%% 4.8 - Validar Anomalías
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=updated_stats_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']
)
context.run(example_validator)
context.show(example_validator.outputs['anomalies'])  # Ver anomalías

NameError: name 'updated_stats_gen' is not defined

In [40]:
#%% 4.9-4.11 - Transformación de Características
from tfx.components import Transform
import tensorflow_transform as tft

def preprocessing_fn(inputs):
    outputs = {}
    # Estandarizar numéricas
    for col in selected_features:
        outputs[col] = tft.scale_to_z_score(inputs[col])
    
    # Conservar binarias
    for col in ['Wilderness_Area_0', 'Soil_Type_0']:
        outputs[col] = inputs[col]
    
    # Etiqueta
    outputs['Cover_Type'] = inputs['Cover_Type']
    
    return outputs

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    preprocessing_fn=preprocessing_fn
)
context.run(transform)

# Inspeccionar salida
print("TransformGraph guardado en:", transform.outputs['transform_graph'].get()[0].uri)

NameError: name 'schema_importer' is not defined